In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy import optimize
import random
import networkx as nx

In [ ]:
# Create network and startsituation (and colors)
def startcolor(nodes, sickPerc, radius, color):
    Network = nx.random_geometric_graph(nodes, radius)
    Situation = []
    Colorlist = []
    for i in range(nodes):
        randomN = random.uniform(0, 1)
        if randomN < sickPerc:
            Situation.append(1)
            Colorlist.append(color[1])
        else:
            Situation.append(0)
            Colorlist.append(color[0])
    return Network, Situation, Colorlist

In [ ]:
## SIR with color
# 0 for S, 1 for I, 2 for R
def updateSIRcolor(y, beta, gamma, Network, color):
    yNew = []
    Colorlist = []
    for i in range(len(Network)):
        if y[i] == 0:
            Connected = Network[i]
            Phealthy = 1
            for j in Connected:
                if y[j] == 1:
                    Phealthy *= (1 - beta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                yNew.append(1)
                Colorlist.append(color[1])
            else:
                yNew.append(y[i])
                Colorlist.append(color[y[i]])
        elif y[i] == 1:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                yNew.append(2)
                Colorlist.append(color[2])
            else:
                yNew.append(y[i])
                Colorlist.append(color[y[i]])
        else:
            yNew.append(y[i])
            Colorlist.append(color[y[i]])
    return yNew, Colorlist

def spreaddiseaseSIRcolor(NLoops, StartSituation, beta, gamma, Network, color, plotnetworks):
    yList = [StartSituation]
    yNew = StartSituation
    pos = nx.get_node_attributes(Network, 'pos')
    for i in range(NLoops):
        y = yNew
        yNew, Colorlist = updateSIRcolor(y, beta, gamma, Network, color)
        yList.append(yNew)
        
        if plotnetworks:
            plt.figure(figsize = (8, 8))
            nx.draw_networkx_edges(Network, pos, alpha= 0.4)
            nx.draw_networkx_nodes(Network, pos, node_size= 100, node_color= Colorlist)
            plt.axis('off')
            plt.show()
    return yList

def countingSIR(situations):
    S = []
    I = []
    R = []
    for i in range(len(situations)):
        S.append(np.bincount(situations[i], minlength = 3)[0])
        I.append(np.bincount(situations[i], minlength = 3)[1])
        R.append(np.bincount(situations[i], minlength = 3)[2])
    return S, I, R

def everythingSIRcolor(nodes, sickPerc, radius, NLoops, beta, gamma, color, plotnetworks):
    Network, StartSituation, Colorlist = startcolor(nodes, sickPerc, radius, color)
    
    if plotnetworks:
        pos = nx.get_node_attributes(Network, 'pos')
        plt.figure(figsize = (8, 8))
        nx.draw_networkx_edges(Network, pos, alpha= 0.4)
        nx.draw_networkx_nodes(Network, pos, node_size= 100, node_color= Colorlist)
        plt.axis('off')
        plt.show()
    
    situations = spreaddiseaseSIRcolor(NLoops, StartSituation, beta, gamma, Network, color, plotnetworks)
    
    S, I, R = countingSIR(situations)
    time = np.arange(0, NLoops + 1, 1)
    Fig_Data = plt.figure()

    plt.plot(time, S, label = 'S', color= color[0])
    plt.plot(time, I, label = 'I', color= color[1])
    plt.plot(time, R, label = 'R', color= color[2])

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12)
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()

    plt.show()
    return Network, S, I, R

def averageSIR(nodes, sickPerc, radius, NLoops, beta, gamma, N, color, plotnetworks):
    time = np.arange(0, NLoops + 1, 1)
    AverageS = np.zeros(NLoops + 1)
    AverageI = np.zeros(NLoops + 1)
    AverageR = np.zeros(NLoops + 1)
    for i in range(N):
        Network, StartSituation, Colorlist = startcolor(nodes, sickPerc, radius, color)
        situations = spreaddiseaseSIRcolor(NLoops, StartSituation, beta, gamma, Network, color, plotnetworks)
        S, I, R = countingSIR(situations)
        AverageS += S
        AverageI += I
        AverageR += R
    AverageS = AverageS/N
    AverageI = AverageI/N
    AverageR = AverageR/N
    
    Fig_Data = plt.figure()

    plt.plot(time, AverageS, label = 'S', color= color[0])
    plt.plot(time, AverageI, label = 'I', color= color[1])
    plt.plot(time, AverageR, label = 'R', color= color[2])

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12)
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()
    
    plt.show()
    return AverageS, AverageI, AverageR

In [ ]:
def ODEsolveI(time, betaf, gammaf):
    def ODE(time, y):
        Spos, Ipos, Rpos = y
        return [-betaf*Ipos*Spos/nodes, betaf*Ipos*Spos/nodes - gammaf*Ipos, gammaf*Ipos]
    solI = solve_ivp(ODE, (time[0], time[-1]), InitialCondition, t_eval = time)
    return solI['y'][1]

In [ ]:
nodes = 300 
sickPerc = 0.1
radius = 0.3 
NLoops = 50
time = np.arange(0, NLoops + 1, 1)
color = ['palegreen', 'tomato', 'deepskyblue']
N = 100
plotnetworks = False
beta = 0.05
gamma = 0.02
InitialCondition= [nodes*(1 - sickPerc), nodes*sickPerc, 0]

AverageS, AverageI, AverageR = averageSIR(nodes, sickPerc, radius, NLoops, beta, gamma, N, color, plotnetworks)

In [ ]:
par, parCov = optimize.curve_fit(ODEsolveI, time, AverageI, p0= [beta, gamma])
print(par)
print('R0 = ', par[0]/par[1])

In [ ]:
Fig_Data = plt.figure()
timeplot = np.arange(0, NLoops + 1, 0.1)
plt.plot(timeplot, ODEsolveI(timeplot, par[0], par[1]), label = 'I fit', color= 'black')
plt.plot(time, AverageI, label= 'I simulation', color= 'tomato')
plt.title('', size= 20)
plt.ylabel('', size= 20)
plt.xlabel('', size= 20)
plt.legend(fontsize= 12, loc = 'upper right')
plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
plt.ylim(0, nodes)
plt.grid()

plt.show()